In [4]:
import pandas as pd
import scipy.io as sio
from scipy import signal, stats
import numpy as np
import datetime
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import re
import coare36vn_zrf_et
import os

riverDF = pd.read_csv('river_temperature12_13.csv', parse_dates = ['DN'])
riverDF['hourDN'] = riverDF['DN'].dt.floor('h')

hourlyMeanRiverDF = riverDF.groupby('hourDN').agg(hourlyTemp=('EstimatedTemp', "mean")).reset_index()
hourlyMeanRiverDF['hourSalinity'] = 0

heatFluxModelDF = pd.read_csv('river12_13_flux.csv', parse_dates = ['Tm (UTC)'])
heatFluxModelDF = pd.merge(left = hourlyMeanRiverDF, right = heatFluxModelDF, how = 'inner',  left_on = 'hourDN', right_on = 'Tm (UTC)')
heatFluxModelDF = heatFluxModelDF.drop('Tm (UTC)', axis = 1)
jdDateBase = pd.Timestamp('1858-11-17').to_julian_date()
heatFluxModelDF['jdTimeStamp'] = heatFluxModelDF['hourDN'].apply(lambda x: x.to_julian_date()- jdDateBase)
heatFluxModelDF = heatFluxModelDF.sort_values(['hourDN'], ascending = [True])
pd.set_option('display.float_format', '{:.20f}'.format)

A=coare36vn_zrf_et.coare36vn_zrf_et(heatFluxModelDF["U (m s-1)"].to_numpy(), 10, heatFluxModelDF["Tair (degC)"].to_numpy(),2, heatFluxModelDF["RH (%)"].to_numpy(),2, heatFluxModelDF["P (mb)"].to_numpy(), heatFluxModelDF["hourlyTemp"].to_numpy(), heatFluxModelDF["SW_dn (W m-2)"].to_numpy(), heatFluxModelDF["LW_dn (W m-2)"].to_numpy(), heatFluxModelDF["Lat"].to_numpy(), heatFluxModelDF["Lon"].to_numpy(),heatFluxModelDF["jdTimeStamp"].to_numpy(),600,0, heatFluxModelDF["hourSalinity"].to_numpy(),None,None,10,10,10)

fnameA = os.path.join("C:\\Users\\nvleb\\OneDrive - Oregon State University\\Desktop\\Python\\AlseaEstuaryData\\Alsea-Estuary-Data",'test_13_output_river_heatflux.csv')
# A=coare36vn_zrf_et(u, zu , t, zt, rh, zq, P, ts, sw_dn, lw_dn, lat, lon,jd, zi,rain, Ss, None , None, zrf_u, zrf_t, zrf_q)
# fnameA = os.path.join(path,'test_36_output_py_082022_withnowavesinput.txt')
A_hdr = 'usr\ttau\thsb\thlb\thbb\thsbb\thlwebb\ttsr\tqsr\tzo\tzot\tzoq\tCd\t'
A_hdr += 'Ch\tCe\tL\tzeta\tdT_skinx\tdq_skinx\tdz_skin\tUrf\tTrf\tQrf\t'
A_hdr += 'RHrf\tUrfN\tTrfN\tQrfN\tlw_net\tsw_net\tLe\trhoa\tUN\tU10\tU10N\t'
A_hdr += 'Cdn_10\tChn_10\tCen_10\thrain\tQs\tEvap\tT10\tT10N\tQ10\tQ10N\tRH10\t'
A_hdr += 'P10\trhoa10\tgust\twc_frac\tEdis'
csvHdr = A_hdr.replace("\t", ",")
np.savetxt(fnameA,A,fmt='%.18e',delimiter=',',header=csvHdr)

processedDataDF = pd.read_csv('test_13_output_river_heatflux.csv')
processedDataDF['Tm (UTC)'] = heatFluxModelDF['hourDN']

processedDataDF["Qnet"] = processedDataDF["sw_net"] + processedDataDF["lw_net"] - processedDataDF["hsb"] - processedDataDF["hlb"]

tempFigLabels = ["sw_net", "lw_net", "hsb", "hlb", "Qnet"]
tempfig = px.line(x = processedDataDF['Tm (UTC)'], y = [processedDataDF["sw_net"], processedDataDF["lw_net"], -1*processedDataDF["hsb"], -1*processedDataDF["hlb"], processedDataDF["Qnet"]], color_discrete_sequence= ["blue", "crimson", "darkorange","green","black"], title = "Heat Flux")
for idx in range(len(tempFigLabels)):
    tempfig.data[idx].name = tempFigLabels[idx]
    tempfig.data[idx].hovertemplate = 'variable=' + tempFigLabels[idx] + '<br>x=%{x}<br>value=%{y}<extra></extra>'
    tempfig.data[idx].legendgroup = tempFigLabels[idx]

tempfig.update_layout(title=dict(text= "Heat Flux for 2013", font=dict(size=25)))
tempfig.update_xaxes(tickangle=30)
tempfig.update_xaxes(rangeslider_visible=True)
tempfig.update_xaxes(range = [pd.Timestamp('2012-12-09'),pd.Timestamp('2013-05-26')])

tempfig.update_layout(xaxis_title="Date", yaxis_title="Heat Flux (W/m2)", legend_title="Location")
tempfig.show()

c:\Users\nvleb\OneDrive - Oregon State University\Desktop\Python\AlseaEstuaryData\Alsea-Estuary-Data\coare36vn_zrf_et.py:750: RuntimeWarning: invalid value encountered in power
  psi = - ((1 + 0.6667 * zeta) ** 1.5 + np.multiply(0.6667 * (zeta - 14.28),np.exp(- dzeta)) + 8.525)


ValueError: The truth value of an empty array is ambiguous. Use `array.size > 0` to check that an array is not empty.